## Implementation of hierarchical multi-label classification networks

This is based on the paper: <i>Hierarchical Multi-Label Classification Networks</i>. Data is from: http://www.biomal.ufscar.br/hmc.html.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Dropout,Activation,Flatten,Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

import warnings
warnings.filterwarnings("ignore")

In [2]:
test_data = pd.read_csv("../data/biology/CellCycle_FUN/cellcycle.test.csv").to_numpy()
train_data = pd.read_csv("../data/biology/CellCycle_FUN/cellcycle.train.csv")
columns = train_data.columns
train_data = train_data.to_numpy()
x_train,y_train = train_data[:,:77],train_data[:,77:]
x_test,y_test = test_data[:,:77],test_data[:,77:]
print(x_train.shape,y_train.shape)

(1628, 77) (1628, 499)


In [3]:
def global_step(h,x):
    """ global optimization step
    """
    h = Concatenate()([x,h])
    h = Dense(50,activation='relu')(h)
    return h

In [4]:
def local_step(x,out_h):
    """ local optimization step
    """
    h = Dense(50,activation="relu")(x)
    out = Dense(out_h,activation="sigmoid")(h)
    return out

In [5]:
def hmcn(b1=0.5,b2=0.5,optimizer=Adam(lr=0.0001)):
    """ hierarchical model
        -penalizing hierarchical violations not included
    """
    x = Input(77)
    
    a = Dense(50,activation='relu')(x)
    p = local_step(a,18)
    
    local_outputs = [p]
    for out_h in [80,178,142,77,4]: # classes per hierarchy
        a = global_step(a,x)
        p = local_step(a,out_h)
        local_outputs.append(p)
    
    local_output = Concatenate()(local_outputs)
    global_output = Dense(499,activation='sigmoid')(a)
    final_output = b1*local_output + b2*global_output
    
    model = Model(inputs=[x],outputs=[final_output])
    model.compile(loss=BinaryCrossentropy(),optimizer=optimizer)
    return model

In [6]:
model = hmcn()

In [7]:
batch_size=20
epochs=10

for epoch_i in range(epochs):
    losses = []
    for i in range(0,len(x_train),batch_size):
        x_train_subset = x_train[i:i+batch_size]
        y_train_subset = y_train[i:i+batch_size]
        batch_loss = model.train_on_batch(x_train_subset,y_train_subset)
        losses.append(float(batch_loss))
        
    test_pred = model(x_test).numpy()
    test_loss = float(BinaryCrossentropy()(y_test,test_pred))
    print("epoch {}; train-loss: {}, test-loss: {}".format(epoch_i+1,round(sum(losses)/len(losses),4),round(test_loss,4)))

epoch 1; train-loss: 0.6652, test-loss: 0.6183
epoch 2; train-loss: 0.5362, test-loss: 0.4169
epoch 3; train-loss: 0.2617, test-loss: 0.149
epoch 4; train-loss: 0.1146, test-loss: 0.0977
epoch 5; train-loss: 0.0879, test-loss: 0.085
epoch 6; train-loss: 0.08, test-loss: 0.0801
epoch 7; train-loss: 0.0764, test-loss: 0.0776
epoch 8; train-loss: 0.0743, test-loss: 0.0761
epoch 9; train-loss: 0.0731, test-loss: 0.0752
epoch 10; train-loss: 0.0722, test-loss: 0.0745
